In [15]:
import numpy as np
def KMeansPlusPlus(data, k):
    n = data.shape[0]
    
    #Initialize the first centroid
    centroids = data[np.random.choice(range(n),1),:]
    
    #broadcast the data sets to calculate the distance conveniently
    data_expand = data[:, np.newaxis, :]
    
    while centroids.shape[0] < k :
        
        #Calculate the distance between 
        pre_distance = (data_expand - centroids) ** 2
        distance = np.sum(pre_distance, axis=2)
        
        #mark the location of minimum distance for every centroid
        min_dist = np.zeros(distance.shape)
        min_dist[range(distance.shape[0]), np.argmin(distance, axis=1)] = 1
        
        #Calculate the normal constant of the distribution
        normal_cons = np.sum(distance[min_dist == 1])
        
        #Get the distribution for sampling new centroids
        distribution = np.min(distance, axis=1)/normal_cons
        
        #Obtain the next centroid and append
        centroids = np.r_[centroids, data[np.random.choice(range(n),1,p=distribution), :]]
    
    return centroids

In [4]:
k = 3
# if all of inputs is correct, it works well.
data1 = np.random.randn(1000,2)
print (KMeansPlusPlus(data1,k))

NameError: name 'KMeansPlusPlus' is not defined

In [13]:
def ScalableKMeansPlusPlus(data, k, l):
    
    centroids = data[np.random.choice(range(data.shape[0]),1), :]
    data_expand = data[:, np.newaxis, :]
    
    #Calculate the iteration times
    pre_distance = (data_expand - centroids) ** 2
    distance = np.sum(pre_distance, axis=2)
    min_dist = np.zeros(distance.shape)
    min_dist[range(distance.shape[0]), np.argmin(distance, axis=1)] = 1
    normal_cons = np.sum(distance[min_dist == 1])
    iter = int(np.log(normal_cons))
    
    for i in range(iter):
        pre_distance = (data_expand - centroids) ** 2
        distance = np.sum(pre_distance, axis=2)
        # the minimum pre_distance
        min = np.min(distance, axis=1).reshape(-1,1)
        # random matrix with the same size as min
        random_matrix = np.random.rand(min.shape[0],min.shape[1])
        # replace zeros in min with the lowest positive float
        min[np.where(min==0)] = np.nextafter(0,1)
        # (1.0/min)th root of random matrix
        random_matrix = random_matrix ** (1.0/min) 
        # choose the highest l
        center = data[np.argsort(random_matrix, axis=0)[:, 0]][::-1][:l, :]
        # combine the new centroids with the old one
        centroids = np.vstack((centroids, center))
        # Finally we get a set of centroids which is higher than k

    ## reduce k*l to k using KMeans++
    pre_distance = (data_expand - centroids) ** 2
    distance = np.sum(pre_distance, axis=2)
    min_dist = np.zeros(distance.shape)
    min_dist[range(distance.shape[0]), np.argmin(distance, axis=1)] = 1
    weights = np.array([np.count_nonzero(min_dist[:, i]) for i in range(centroids.shape[0])]).reshape(-1,1)
    prob_distribution = weights/np.sum(weights)
    centroids = data[np.random.choice(range(weights.shape[0]),k,p=prob_distribution.ravel()),:]
    
    return centroids


In [14]:
k = 3
l = 2
data1 = np.random.randn(1000,2)
ScalableKMeansPlusPlus(data1,k,l)

array([[-0.30352925, -0.50441264],
       [-1.22870384, -0.08620885],
       [ 0.56440645,  0.30014885]])